# The Critics Critique ML model 1

## 0. import libraries

In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


## 1. import and slice dataset 

In [2]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [3]:
len(all_sites)

15551

In [4]:
all_sites1 = all_sites[:3100]

In [5]:
all_sites2 = all_sites[3100:6200]

In [6]:
all_sites3 = all_sites[6200:9300]

In [7]:
all_sites4 = all_sites[9300:1240]

In [8]:
all_sites5 = all_sites[12400:]

## 2.2 Classifier with Transformers nlptown/bert-base-multilingual-uncased-sentiment

In [5]:
# This let us see prob for every batch 1-5

'''from adaptnlp import EasySequenceClassifier

classifier = EasySequenceClassifier()

sentences = classifier.tag_text(
    text=text_splitted,
    model_name_or_path="nlptown/bert-base-multilingual-uncased-sentiment",
    padding='longest',
    mini_batch_size=4
)

print("Tag Score Outputs:\n")
for sentence in sentences:
    print(f"Fragment {sentences.index(sentence)}: {sentence.labels}")'''

'from adaptnlp import EasySequenceClassifier\n\nclassifier = EasySequenceClassifier()\n\nsentences = classifier.tag_text(\n    text=text_splitted,\n    model_name_or_path="nlptown/bert-base-multilingual-uncased-sentiment",\n    padding=\'longest\',\n    mini_batch_size=4\n)\n\nprint("Tag Score Outputs:\n")\nfor sentence in sentences:\n    print(f"Fragment {sentences.index(sentence)}: {sentence.labels}")'

### Sentiment proccesing function Bert base Multilingual Uncased 

In [5]:
# Select model
nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(nlp_model)
model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

# Classifier
classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer)

In [9]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 1500
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    global_stars = (classifier(review_splitted))
    
    return global_stars


In [10]:
def apply_sentiment_model(df):
    try:
        df['stars'] = df['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))
        
    except RuntimeError:
        print(df['game'])
        df['stars'] = df['text'].apply(lambda x: np.nan)
    
    finally:
        df.to_csv('../data/labeled_texts1.csv', index=False)
    
    return df

In [11]:
apply_sentiment_model(all_sites1)

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,site,url_link,author,game,company,genre,platform,text,score,stars
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5,"[{'label': '3 stars', 'score': 0.4069659113883..."
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2,"[{'label': '4 stars', 'score': 0.4059227108955..."
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0,"[{'label': '3 stars', 'score': 0.4342378377914..."
3,meristation,https://as.com/meristation/2014/10/06/analisis...,Salva Fernàndez,Titanfall: IMC Rising,None,"Acción, First Person Shooter",PC 360 XBO,Titanfall recibe el último contenido descargab...,7.0,"[{'label': '3 stars', 'score': 0.4690837860107..."
4,meristation,https://as.com/meristation/2014/04/16/analisis...,None,Moebius: Empire Rising,None,Aventura Gráfica,PC OSX IPH IPD AND,"“Aquellos que no pueden recordar el pasado, es...",6.8,"[{'label': '4 stars', 'score': 0.4857680797576..."
...,...,...,...,...,...,...,...,...,...,...
3095,3D Juegos,https://www.3djuegos.com/juegos/analisis/9035/...,Alberto Pastor,Animal Crossing: New Leaf - Análisis,None,None,3DS,Con una nueva filosofía de juego pero el mismo...,9.0,"[{'label': '5 stars', 'score': 0.7128560543060..."
3096,meristation,https://as.com/meristation/2012/01/23/analisis...,Cristian Ciuraneta,Unstoppable Gorg,Futuremark Games Studio,"Strategy, Arcade",PC IPD,Los amantes de la ciencia ficción de mediados ...,7.5,"[{'label': '3 stars', 'score': 0.4269151091575..."
3097,3D Juegos,https://www.3djuegos.com/juegos/analisis/11454...,Sergio Martín,UFC Undisputed 3 - Análisis,None,None,PS3 X360,La saga de lucha extrema UFC (ahora Undisputed...,8.6,"[{'label': '5 stars', 'score': 0.5908300280570..."
3098,meristation,https://as.com/meristation/2018/06/07/analisis...,Sergio C. González,Sushi Striker: The Way of the Sushido,Nintendo,"Acción, Puzzle",NSW 3DS,Nintendo no acostumbra a desperdiciar ideas. L...,7.0,"[{'label': '2 stars', 'score': 0.4140197634696..."


In [10]:
'''def sentiment_analysis_ind(df):
    global_stars = []
    
    # Select model
    nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(nlp_model)
    model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

    # Classifier
    classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer
    )
    
    # Adapt review lenght to max tokens
    n = 1900
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]
    #print(len(review_splitted))
    
    # Classifying 
    for text in review_splitted:
            stars = (classifier(text))
            global_stars.append(stars)

    return global_stars'''

### Sentiment Synthetic Score Function

In [12]:
def stars_mean_to_score(review):
    
    points = []

    for classification in review:

        #Mean of fragment
        grade = int(classification['label'].split(' ')[0])
        points.append(grade)  
    score = round(np.mean(points), 2)

    return(score)   

In [13]:
all_sites1['stars_mean'] = all_sites1['stars'].apply(lambda x: stars_mean_to_score(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
len(all_sites1)

3100

### Score to Stars Equivalence

In [17]:
def score_labels(score):
    '''Score converted to labels 1-5 and rounded'''
    
    scorelabel = f"'{round(score / 2)} starts'"
    return scorelabel

In [18]:
all_sites1['score_label'] = all_sites1['score'].apply(lambda x: score_labels(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Score Adjusted

In [19]:
def score_half(score):
    '''Score reduced to compare with stars mean'''
    return score / 2

In [20]:
all_sites1['score_adj'] = all_sites1['score'].apply(lambda x: score_half(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Export full dataframe

In [21]:
all_sites1.to_csv('../data/scored_texts1.csv', index=False)

In [22]:
all_sites1

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean,score_label,score_adj
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5,"[{'label': '3 stars', 'score': 0.4069659113883...",3.17,'4 starts',4.25
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2,"[{'label': '4 stars', 'score': 0.4059227108955...",4.00,'4 starts',3.60
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0,"[{'label': '3 stars', 'score': 0.4342378377914...",3.62,'4 starts',4.00
3,meristation,https://as.com/meristation/2014/10/06/analisis...,Salva Fernàndez,Titanfall: IMC Rising,None,"Acción, First Person Shooter",PC 360 XBO,Titanfall recibe el último contenido descargab...,7.0,"[{'label': '3 stars', 'score': 0.4690837860107...",3.40,'4 starts',3.50
4,meristation,https://as.com/meristation/2014/04/16/analisis...,None,Moebius: Empire Rising,None,Aventura Gráfica,PC OSX IPH IPD AND,"“Aquellos que no pueden recordar el pasado, es...",6.8,"[{'label': '4 stars', 'score': 0.4857680797576...",3.08,'3 starts',3.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,3D Juegos,https://www.3djuegos.com/juegos/analisis/9035/...,Alberto Pastor,Animal Crossing: New Leaf - Análisis,None,None,3DS,Con una nueva filosofía de juego pero el mismo...,9.0,"[{'label': '5 stars', 'score': 0.7128560543060...",3.82,'4 starts',4.50
3096,meristation,https://as.com/meristation/2012/01/23/analisis...,Cristian Ciuraneta,Unstoppable Gorg,Futuremark Games Studio,"Strategy, Arcade",PC IPD,Los amantes de la ciencia ficción de mediados ...,7.5,"[{'label': '3 stars', 'score': 0.4269151091575...",3.50,'4 starts',3.75
3097,3D Juegos,https://www.3djuegos.com/juegos/analisis/11454...,Sergio Martín,UFC Undisputed 3 - Análisis,None,None,PS3 X360,La saga de lucha extrema UFC (ahora Undisputed...,8.6,"[{'label': '5 stars', 'score': 0.5908300280570...",4.00,'4 starts',4.30
3098,meristation,https://as.com/meristation/2018/06/07/analisis...,Sergio C. González,Sushi Striker: The Way of the Sushido,Nintendo,"Acción, Puzzle",NSW 3DS,Nintendo no acostumbra a desperdiciar ideas. L...,7.0,"[{'label': '2 stars', 'score': 0.4140197634696...",3.43,'4 starts',3.50


## 3 Fine Tuning BERT model classifier

Train the entire architecture. Train some layers while freezing others, Freeze the entire architecture https://huggingface.co/transformers/training.html

### 3.1 Get labels

In [99]:
all_test_small['stars_mean'] = all_test_small['stars'].apply(lambda x: stars_mean_to_score(x))

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean,score_label
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5,"[[{'label': '3 stars', 'score': 0.435262888669...",3.60,'4 starts'
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2,"[[{'label': '3 stars', 'score': 0.421717613935...",3.20,'4 starts'
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0,"[[{'label': '3 stars', 'score': 0.415306359529...",3.50,'4 starts'
3,meristation,https://as.com/meristation/2014/10/06/analisis...,Salva Fernàndez,Titanfall: IMC Rising,None,"Acción, First Person Shooter",PC 360 XBO,Titanfall recibe el último contenido descargab...,7.0,"[[{'label': '3 stars', 'score': 0.483339041471...",3.25,'4 starts'
4,meristation,https://as.com/meristation/2014/04/16/analisis...,None,Moebius: Empire Rising,None,Aventura Gráfica,PC OSX IPH IPD AND,"“Aquellos que no pueden recordar el pasado, es...",6.8,"[[{'label': '4 stars', 'score': 0.495593637228...",3.20,'3 starts'


### 3.2 With Trainer
https://huggingface.co/transformers/custom_datasets.html

In [72]:
#model.train()

In [91]:
full_texts = all_test_small['text']

In [92]:
full_labels = all_test_small['score_label']

In [93]:
train_texts, val_texts, train_labels, val_labels = train_test_split(full_texts, full_labels, test_size=.2)

In [94]:
val_texts = val_texts.astype(str).tolist()
train_texts = train_texts.astype(str).tolist()

In [76]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [95]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [96]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

In [81]:
from transformers import Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset            # evaluation dataset
)

In [82]:
trainer.train()
#trainer.evaluate()

KeyError: 11

### 3.3 With Torch

In [97]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

TypeError: new(): invalid data type 'str'